In [ ]:
import exodus3 as exo
import numpy as np
import os.path

In [ ]:
newBlocks = (0,2,10,30)
def blockindicator(xc,yc):
    if xc < 0.0:
        if yc <= 0.0:
            return 0
        else:
            return 2
    else:
        if yc <= 0.0:
            return 10
        else:
            return 30


In [ ]:
infile = '../TestMeshes/SquareNG_X.gen'
infile = '../TestMeshes/SquareFaceSetCubit2CS.gen'
outfile = '../TestMeshes/SquareNG_X_Blocks.gen'
e = exo.exodus(infile,array_type='numpy')

In [ ]:
# check that all cells are of the same type
nv = 0
cellType = None
numCells = 0
numAttr  = 0
for blk in e.get_elem_blk_ids():
    cellType, ncBlk, nvBlk, naBlk = e.elem_blk_info(blk)
    tmpConnect = e.get_elem_connectivity(blk)
    numCells += ncBlk
    if nv == 0:
        nv = nvBlk
    elif not nv == nvBlk:
        print('Error: all blocks must have the same cell type')
    if not cellType:
        cellType = cellType
    elif not cellType == cellType:
        print('Error: all blocks must have the same cell type')
    if numAttr == 0:
        numAttr = naBlk
    elif not numAttr == naBlk:
        print('Error: all blocks must have the same number of attributes')
print("number of vertex per cells: {}".format(nv))
print("Total number of cells: {}".format(numCells))
print("cell type: {}".format(cellType))
print("Number of cell attributes {}".format(numAttr))


In [ ]:
# read the entire connectivity table in one array (this could be optimized)
connect = np.zeros(nv*numCells,dtype=np.int32)
offset = 0
for blk in e.get_elem_blk_ids():
    tmpConnect = e.get_elem_connectivity(blk)
    connect[offset:offset+tmpConnect[1]*nv] = tmpConnect[0]
    offset += tmpConnect[1]*nv

In [ ]:
# reconstruct connectivity tables for each block
newConnect = {id:np.empty(0,dtype=np.int32) for id in newBlocks}
x,y,z = e.get_coords()
for blk in e.get_elem_blk_ids():
    EXOconnect = e.get_elem_connectivity(blk)
    for n in range(EXOconnect[1]):
        cell = EXOconnect[0][n*nv:(n+1)*nv]
        xc = np.average(x[cell-1])
        yc = np.average(y[cell-1])
        blkID = blockindicator(xc,yc)
        newConnect[blkID] = np.append(newConnect[blkID],cell)

In [ ]:
# write the output file
if os.path.exists(outfile):
    os.remove(outfile)
f = exo.exodus(outfile,mode = 'w', array_type='numpy',
               title = e.title(),
               numDims = e.numDim.value, 
               numNodes = e.numNodes.value, 
               numElems = e.numElem.value,
               numBlocks = len(newBlocks), 
               numNodeSets = e.numNodeSets.value, 
               numSideSets = e.numSideSets.value,
               io_size  = 0)


In [ ]:
f.put_coord_names(["x", "y"])
f.put_coords(x,y,z)

vsid = e.get_node_set_ids()
for vs in vsid:
    numSetNodes, numSetDistFacts = e.get_node_set_params(vs)
    f.put_node_set_params(vs,numSetNodes, numSetDistFacts)

fsid = e.get_side_set_ids()
for fs in fsid:
    fs_elems, fs_sides = e.get_side_set_params(fs)
    f.put_side_set_params(fs,fs_elems, fs_sides)

for blk in newBlocks:
    f.put_elem_blk_info(blk, cellType, len(newConnect[blk])//nv, nv, numAttr)
    f.put_elem_connectivity(blk, newConnect[blk])

In [ ]:
f.close()